In [ ]:
#@title 🤗 AutoTrain LLM
#@markdown In order to use this colab
#@markdown - upload train.csv to a folder named `data/`
#@markdown - train.csv must contain a `text` column
#@markdown - choose a project name if you wish
#@markdown - change model if you wish, you can use most of the text-generation models from Hugging Face Hub
#@markdown - add huggingface information (token) if you wish to push trained model to huggingface hub
#@markdown - update hyperparameters if you wish
#@markdown - click `Runtime > Run all` or run each cell individually
#@markdown - report issues / feature requests here: https://github.com/huggingface/autotrain-advanced/issues


import os
!pip install -U autotrain-advanced > install_logs.txt 2>&1
!autotrain setup --colab > setup_logs.txt
from autotrain import __version__
print(f'AutoTrain version: {__version__}')

In [1]:
import os
os.makedirs("data", exist_ok=True)

In [2]:
from google.colab import files
uploaded = files.upload()  # upload your .jsonl here

Saving ariel_text_format.jsonl to ariel_text_format (1).jsonl


In [22]:
import os
import json
import pandas as pd

# Make sure the data folder exists
os.makedirs("data", exist_ok=True)

# Load your file
with open("ariel_text_format.jsonl", "r", encoding="utf-8") as f:
    rows = [json.loads(line) for line in f]

# Create DataFrame with "text" column
df = pd.DataFrame(rows)

# Confirm it's correct
print(df.head(3))

# Save to CSV
df.to_csv("data/train.csv", index=False)

print("✅ Converted to data/train.csv with full User + Ariel messages.")



                                                text
0  User: they are nice. try going to nordstrom.co...
1                    User: u there?\nYoung Ariel: ye
2                     User: oh hai\nYoung Ariel: hi*
✅ Converted to data/train.csv with full User + Ariel messages.


In [23]:
import pandas as pd

df = pd.read_csv("data/train.csv")
print(df.head(5).to_string())



                                                                                                                      text
0  User: they are nice. try going to nordstrom.com again they might have new stuff\nYoung Ariel: the second one was irit's
1                                                                                          User: u there?\nYoung Ariel: ye
2                                                                                           User: oh hai\nYoung Ariel: hi*
3                                                              User: i see you've got the KONY stuff ...\nYoung Ariel: yeh
4                                                                                    User: im in israel\nYoung Ariel: nice


In [7]:
#@markdown ---
#@markdown #### Project Config
#@markdown Note: if you are using a restricted/private model, you need to enter your Hugging Face token in the next step.
project_name = 'young-ariel' # @param {type:"string"}
model_name = 'tiiuae/falcon-rw-1b' # @param {type:"string"}

#@markdown ---
#@markdown #### Push to Hub?
#@markdown Use these only if you want to push your trained model to a private repo in your Hugging Face Account
#@markdown If you dont use these, the model will be saved in Google Colab and you are required to download it manually.
#@markdown Please enter your Hugging Face write token. The trained model will be saved to your Hugging Face account.
#@markdown You can find your token here: https://huggingface.co/settings/tokens
push_to_hub = False # @param ["False", "True"] {type:"raw"}
hf_token = "hf_XXX" #@param {type:"string"}
hf_username = "abc" #@param {type:"string"}

#@markdown ---
#@markdown #### Hyperparameters
unsloth = False # @param ["False", "True"] {type:"raw"}
learning_rate = 2e-4 # @param {type:"number"}
num_epochs = 2 #@param {type:"number"}
batch_size = 1 # @param {type:"slider", min:1, max:32, step:1}
block_size = 1024 # @param {type:"number"}
trainer = "sft" # @param ["generic", "sft"] {type:"string"}
warmup_ratio = 0.1 # @param {type:"number"}
weight_decay = 0.01 # @param {type:"number"}
gradient_accumulation = 4 # @param {type:"number"}
mixed_precision = "fp16" # @param ["fp16", "bf16", "none"] {type:"string"}
peft = True # @param ["False", "True"] {type:"raw"}
quantization = "int4" # @param ["int4", "int8", "none"] {type:"string"}
lora_r = 16 #@param {type:"number"}
lora_alpha = 32 #@param {type:"number"}
lora_dropout = 0.05 #@param {type:"number"}

os.environ["HF_TOKEN"] = hf_token
os.environ["HF_USERNAME"] = hf_username

conf = f"""
task: llm-{trainer}
base_model: {model_name}
project_name: {project_name}
log: tensorboard
backend: local

data:
  path: data/
  train_split: train
  valid_split: null
  chat_template: null
  column_mapping:
    text_column: text

params:
  block_size: {block_size}
  lr: {learning_rate}
  warmup_ratio: {warmup_ratio}
  weight_decay: {weight_decay}
  epochs: {num_epochs}
  batch_size: {batch_size}
  gradient_accumulation: {gradient_accumulation}
  mixed_precision: {mixed_precision}
  peft: {peft}
  quantization: {quantization}
  lora_r: {lora_r}
  lora_alpha: {lora_alpha}
  lora_dropout: {lora_dropout}
  unsloth: {unsloth}

hub:
  username: ${{HF_USERNAME}}
  token: ${{HF_TOKEN}}
  push_to_hub: {push_to_hub}
"""

with open("conf.yaml", "w") as f:
  f.write(conf)

In [28]:
!autotrain --config conf.yaml


INFO     | 2025-06-19 16:48:13 | autotrain.cli.run_llm:run:136 - Running LLM
WARNING  | 2025-06-19 16:48:13 | autotrain.trainers.common:__init__:286 - Parameters supplied but not used: inference, deploy, config, backend, version, func, train
Saving the dataset (1/1 shards): 100% 15993/15993 [00:00<00:00, 1402837.98 examples/s]
Saving the dataset (1/1 shards): 100% 15993/15993 [00:00<00:00, 2287841.20 examples/s]
INFO     | 2025-06-19 16:48:13 | autotrain.backends.local:create:20 - Starting local training...
INFO     | 2025-06-19 16:48:13 | autotrain.commands:launch_command:514 - ['accelerate', 'launch', '--num_machines', '1', '--num_processes', '1', '--mixed_precision', 'fp16', '-m', 'autotrain.trainers.clm', '--training_config', 'young-ariel/training_params.json']
INFO     | 2025-06-19 16:48:13 | autotrain.commands:launch_command:515 - {'model': 'tiiuae/falcon-rw-1b', 'project_name': 'young-ariel', 'data_path': 'young-ariel/autotrain-data', 'train_split': 'train', 'valid_split': None,

In [9]:
!pip install -U autotrain-advanced

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 123.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.6/51.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.3/341.3 kB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.9/269.9 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.5/225.5 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 12.2 MB/s eta 0